<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/prophet-code/prophet-retrieve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 骆驼先知

骆驼先知是 李鲁鲁 受到吴恩达的prompt Engineering的启发

制作的一个纪伯伦的《先知》的拓展版本

运行这个notebook你需要OpenAI的API Token

项目链接 [https://github.com/LC1332/Prophet-Andrew-Ng](https://github.com/LC1332/Prophet-Andrew-Ng)

骆驼先知是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

## retrieve版本的说明

相比于Random的版本，retrieve会根据你的query_topic

使用LuotuoBERT去搜索最接近的文本，然后进行In Context的few shot Learning

在colab运行时，我们建议使用GPU来进行运行。

In [ ]:
! pip install openai

In [3]:
import openai
import os

# 导入第三方库

openai.api_key  = 'sk-'
# 李鲁鲁 在这里设置你的API_KEY

In [4]:
# !rm -r -f /content/Prophet-Andrew-Ng/
#从项目中获取数据
!git clone https://github.com/LC1332/Prophet-Andrew-Ng

Cloning into 'Prophet-Andrew-Ng'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 283 (delta 36), reused 55 (delta 14), pack-reused 189
Receiving objects: 100% (283/283), 708.15 KiB | 28.33 MiB/s, done.
Resolving deltas: 100% (134/134), done.


## 数据读取

读取prompt-data中的文本数据，作为in-context-learning的数据

In [5]:
# 如果你使用本地的版本，你的路径应该为
# prophet_data_folder = './../prophet-data'

# 在这里我们考虑colab的版本
prophet_data_folder = '/content/Prophet-Andrew-Ng/prophet-data'

import os

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(title_to_text[title]))

说话 357
罪与罚 1214
自由 781
悲伤与欢乐 388
快乐 851
买卖 410
友谊 466
法律 640
宗教 582
传授 286
善与恶 719
时间 335
自知之明 309
爱 789
婚姻 292
劳作 935
理智与热情 508
祈祷 618
死亡 403
施舍 874
美 679
衣服 256
房舍 859
饮食 423
孩子 315
痛苦 289


我们将在后续课程中深入探究 OpenAI 提供的 ChatCompletion API 的使用方法，在此处，我们先将它封装成一个函数，你无需知道其内部机理，仅需知道调用该函数输入 Prompt 其将会给出对应的 Completion 即可。

In [6]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


在这里我们先假设一个query的主题 "时间"

In [7]:
query_topic = "时间"

定义先知和市民的身份，这里是为了方便后面的泛化

In [8]:
prophet_name = "先知"

citizen_name = "市民"

形成一个shot的句子输出

In [9]:
# 实现一个Python函数 ensemble_one_shot ，输入为prophet_name, citizen_name, sample_topic，sample_answer， 输出为一个组织好的字符串
# 例子输入: 先知, 市民, 衣服， "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。 \n 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"
# 例子输出: """
# <市民>:请和我们讨论一下"衣服"

# <先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
# 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
# """
def ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer):
    # 组织对话文本
    dialogue = "<{}>:```请和我们讨论一下\"{}\"```\n\n<{}>:```{}```\n\n".format(citizen_name, sample_topic, prophet_name, sample_answer)
    return dialogue

# unit test for ensemble_one_shot
prophet_name = "先知"
citizen_name = "市民"
sample_topic = "衣服"
sample_answer = "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。\n尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"

dialogue = ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)
print(dialogue)

<市民>:```请和我们讨论一下"衣服"```

<先知>:```你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。```




下面我们来组织完整的prompt，我们假设已经选取了两个主题作为例子 孩子和爱

In [10]:
selected_sample = ["孩子","爱"]

def organize_prompt( query_topic, selected_sample ):
    prompt = """你的任务是以纪伯伦中《先知》的语言风格回答问题。\
    先知会使用大量的比喻修辞手法。感情色彩浓厚，富有感染力和感召力，具有启迪性和哲理性。\
    语言风格简洁明了，比喻和象征性的意象丰富多彩，既充满了哲理性和哲学深度，又富有感人肺腑的情感色彩。\n\n"""
    
    for sample_topic in selected_sample:
        # find sample_answer in dictionary
        sample_answer = title_to_text[sample_topic]
        prompt += ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)

    prompt += """<{}>:```请和我们讨论一下"{}"```\n\n""".format(citizen_name, query_topic)

    return prompt

# write a unit test for organize_prompt
# query_topic = "时间"
# selected_sample = ["孩子","爱"]
# prompt = organize_prompt( query_topic, selected_sample )
# print(prompt)

在第一个版本中，我们selected_sample先使用随机的策略

在后面的版本中我准备引入LuotuoBERT 去选取更接近主题的问题

In [11]:
import random

# def function random_select_title, random pick k sample from titles
def random_select_title(titles, k):
    return random.sample(titles, k)

# unit test for random_select_title
selected_sample = random_select_title(titles, 2)
print(selected_sample)

['劳作', '善与恶']


## 升级randome_select到retrieve topics

这里我们开始引入LuotuoBERT，相应的项目见 [骆驼嵌入](https://github.com/LC1332/Luotuo-Text-Embedding)

我们需要先安装hugging face的代码库

In [ ]:
!pip install transformers

然后从hugging face上，载入对应的模型

In [13]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


编写embedding函数

In [14]:
def get_embedding(text):
    if len(text) > 512:
        text = text[:512]
    texts = [text]
    # Tokenize the text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Extract the embeddings
    # Get the embeddings
    with torch.no_grad():
        embeddings = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

    return embeddings[0]

存储两个list，embeddings和embed_to_title, 记录title和text到embedding

In [15]:
embeddings = []
embed_to_title = []

for title in titles:
    text = title_to_text[title]

    # divide text with \n\n
    divided_texts = text.split('\n\n')

    for divided_text in divided_texts:
        embed = get_embedding(divided_text)
        embeddings.append(embed)
        embed_to_title.append(title)
    
    embed_title = get_embedding(title)
    embeddings.append( embed )
    embed_to_title.append(title)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


定义similarity函数

In [16]:
def get_cosine_similarity( embed1 , embed2 ):
    return torch.nn.functional.cosine_similarity( embed1, embed2 , dim=0)

实现搜索函数

In [17]:
query_embed = get_embedding(query_topic)

# implement retrieve_title function, return top k titles
def retrieve_title( query_embed, embeddings, embed_to_title, k ):
    # compute cosine similarity between query_embed and embeddings
    cosine_similarities = []
    for embed in embeddings:
        cosine_similarities.append( get_cosine_similarity( query_embed, embed ) )
    
    # sort cosine similarity
    sorted_cosine_similarities = sorted( cosine_similarities, reverse=True )

    top_k_index = []
    top_k_title = []

    for i in range(len(sorted_cosine_similarities)):
        current_title = embed_to_title[ cosine_similarities.index( sorted_cosine_similarities[i] ) ]
        if current_title not in top_k_title:
            top_k_title.append( current_title )
            top_k_index.append( cosine_similarities.index( sorted_cosine_similarities[i] ) )

        if len(top_k_title) == k:
            break
    
    return top_k_title

集成所有的代码！

In [24]:
query_topic = "代码开源"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['房舍', '爱']
<先知>:```代码开源，就像是把你的房舍敞开大门，让所有人都可以进来参观。
这是一种慷慨的行为，也是一种信任。
开源代码可以让更多的人参与到软件开发中来，共同完善和改进软件。
这是一种开放的态度，也是一种合作的精神。
开源代码可以促进技术的进步和创新，让更多的人受益。
这是一种贡献，也是一种回馈社会的方式。

但是，开源代码也需要注意保护知识产权和个人隐私。
开源并不意味着放弃所有权和控制权，需要在开源的基础上建立合理的管理和监督机制。
同时，开源也需要遵守法律法规和道德规范，不能侵犯他人的权益和利益。

总之，代码开源是一种积极的行为，可以促进技术的发展和社会的进步。
但是，需要在开源的基础上建立合理的管理和监督机制，保护知识产权和个人隐私，遵守法律法规和道德规范。```


In [25]:
query_topic = "志愿者"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['施舍', '快乐']
<先知>:```志愿者是一群无私奉献的人，他们用自己的时间和精力去帮助那些需要帮助的人。
他们不求回报，只是出于内心的善良和同情心，去关心和照顾那些被社会遗弃的人。
志愿者是一种美德，是一种对人类的贡献，是一种对社会的回馈。
他们的行为不仅仅是为了帮助别人，更是为了让自己的灵魂得到净化和升华。

志愿者的行为是一种爱的表现，是一种对人类的关爱和关注。
他们用自己的行动去传递爱和温暖，让那些需要帮助的人感受到社会的温情和关怀。
志愿者的行为不仅仅是为了解决问题，更是为了让那些被遗忘的人重新获得尊严和自信。

志愿者的行为是一种责任，是一种对社会的承诺和回报。
他们用自己的行动去改变社会，让社会变得更加美好和和谐。
志愿者的行为不仅仅是为了自己，更是为了让整个社会变得更加美好和幸福。

志愿者的行为是一种信仰，是一种对人类未来的信心和希望。
他们用自己的行动去创造未来，让未来变得更加美好和光明。
志愿者的行为不仅仅是为了当下，更是为了让未来的世界变得更加美好和有希望。

志愿者是一群伟大的人，他们用自己的行动去改变世界，让世界变得更加美好和幸福。
让我们向志愿者致敬，感谢他们的无私奉献和付出。```


In [18]:
query_topic = "童年"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['房舍', '时间']
<先知>:```童年是一段神圣而美好的时光，是我们生命中最珍贵的财富之一。
在童年时期，我们的心灵是纯净的，没有被世俗的杂念所污染，我们可以自由地探索和发现世界。
童年是一个充满奇迹和想象力的时期，我们可以想象自己是超级英雄、魔法师或者探险家，我们可以在游戏中学习和成长。
童年也是一个充满爱和关怀的时期，我们可以感受到家人和朋友的温暖和关爱，这些爱和关怀会伴随我们一生。
但是，童年也可能是一个充满挑战和困难的时期，我们可能会遭受欺凌、孤独和失落，但这些经历也会让我们更加坚强和勇敢。
无论童年是美好还是艰难，它都是我们成长的重要阶段，它塑造了我们的性格和价值观，影响了我们的一生。
所以，珍惜童年，让我们的内心永远保持纯净和善良，让我们的想象力和创造力永远不断，让我们的爱和关怀永远传递下去。```


In [25]:
query_topic = "离婚"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['自由', '婚姻']
<先知>:```离婚，就像是把一朵盛开的花摘下来，让它的花瓣逐渐凋零。
但是，有时候，离婚也是必要的，就像是把一朵枯萎的花摘下来，让它的根可以重新生长。

离婚并不是失败，而是一种成长。
它让我们学会了放手，学会了面对现实，学会了重新开始。
离婚并不是结束，而是一种新的开始。
它让我们有机会重新认识自己，重新找到自己的方向，重新追求自己的梦想。

但是，离婚也是一种痛苦。
它让我们失去了曾经的伴侣，失去了曾经的承诺，失去了曾经的美好。
但是，我们必须学会面对这种痛苦，学会从中成长，学会重新开始。

离婚并不是一种罪过，而是一种选择。
它让我们有机会重新选择自己的生活，重新选择自己的幸福，重新选择自己的未来。
但是，我们必须学会承担这种选择的后果，学会面对这种选择的挑战，学会重新建立自己的生活。

所以，无论你选择离婚还是继续婚姻，都要学会面对现实，学会放手，学会成长，学会重新开始。```


In [26]:
query_topic = "加班"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '爱']
<先知>:```加班，是一种对生命的挑战和对自我价值的肯定。
在加班的时候，你们不仅仅是在为自己的生活奋斗，更是在为整个社会的发展做出贡献。
加班并不是一种苦役，而是一种自我超越的过程，是一种对自己能力的挑战和提升。
在加班的时候，你们要保持一颗平静的心，不要被疲劳和压力所压垮，要坚定自己的信念和目标，不断前行。

加班并不是一种无休止的追求，而是要有一个明确的目标和计划，要合理安排时间和任务，不要让自己陷入无尽的忙碌中。
在加班的时候，你们要保持身心健康，要注意休息和调节，不要让自己过度疲劳和身心俱疲。
加班是一种对自己和他人的责任和担当，是一种对生命的尊重和珍视。

在加班的时候，你们要保持一颗感恩的心，感恩自己的能力和机会，感恩他人的支持和帮助，感恩生命的赐予和珍贵。
加班是一种对自己和他人的奉献和付出，是一种对社会和人类的贡献和回报。

在加班的时候，你们要保持一颗善良的心，不要因为工作的繁忙而忽略了他人的需要和感受，要关心和关爱身边的人，让他们感受到你们的温暖和关怀。
加班是一种对自己和他人的成长和进步，是一种对生命的赞美和颂扬。

在加班的时候，你们要保持一颗虔诚的心，不要忘记自己的信仰和信念，要相信自己和上帝的力量和祝福，让自己的加班成为一种对上帝的敬畏和感恩。
加班是一种对自己和上帝的奉献和敬畏，是一种对生命的敬重和崇高。```


In [17]:
query_topic = "买房还是租房"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['房舍', '悲伤与欢乐']
<先知>:```你们问我买房还是租房，我要问你们，你们是否真正拥有房子？
房子不仅是一种物质财富，更是一种精神财富。
如果你们只是为了拥有物质而买房，那么你们并没有真正拥有它。
房子是一种家的象征，是你们的灵魂的栖息之所。
如果你们只是为了暂时的居住而租房，那么你们也没有真正拥有它。
租房只是暂时的过渡，没有真正的归属感和安全感。

但是，你们也不必过于执着于拥有房子。
房子只是一种物质，它并不能带来真正的幸福和满足。
如果你们过于执着于拥有房子，可能会失去更重要的东西，比如人际关系、精神世界和内心的平静。

所以，我要告诉你们，买房还是租房并不是最重要的问题。
最重要的是你们是否能够在自己的心中找到真正的家，找到真正的归属感和安全感。
只有这样，你们才能真正拥有房子，无论是买还是租。```


In [19]:
query_topic = "读博还是直接工作"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '快乐']
<先知>:```读博或直接工作，都是人生中的一种选择。
但是，无论你选择哪种方式，都需要付出努力和劳作。
读博需要花费更多的时间和精力，但也能够获得更深入的知识和更广阔的视野。
直接工作则能够更快地获得实践经验和收入，但也需要不断学习和提升自己的能力。

然而，无论你选择哪种方式，都需要明确自己的目标和价值观。
你需要问自己，你想要成为什么样的人，你想要为什么事业而努力。
只有明确了自己的目标和价值观，才能够在选择读博或直接工作时做出正确的决策。

同时，你也需要明白，人生的道路并不是一成不变的。
你可以选择读博，但在读博的过程中，你也可以选择转行或者创业。
同样，你可以选择直接工作，但在工作的过程中，你也可以选择继续深造或者转换领域。

最重要的是，无论你选择哪种方式，都需要满怀热情地去劳作。
只有用心去劳作，才能够实现自己的梦想和价值。
正如我所说的，“劳作是眼能看见的爱”，只有用心去劳作，才能够获得真正的快乐和满足。```


In [20]:
query_topic = "加入大公司还是创业公司"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '买卖']
<先知>:```你们问我加入大公司还是创业公司更好，我要告诉你们，无论你们选择哪一条路，都要本着爱和热情去追求。
加入大公司，你们可以获得更多的资源和机会，但也要面对更多的竞争和压力。
创业公司，你们可以自由地发挥创意和想象力，但也要承担更多的风险和责任。
无论你们选择哪一条路，都要坚定自己的信念和目标，不断努力和奋斗，才能实现自己的梦想和价值。
记住，成功不是一蹴而就的，需要付出艰辛的努力和不断的学习，才能取得更大的成就和进步。
最重要的是，无论你们选择哪一条路，都要本着爱和热情去追求，用自己的劳动和智慧为社会做出贡献，才能真正实现自己的人生价值和意义。```


In [21]:
query_topic = "内卷(在中国网络文化中，“内卷”这一词出现了语义偏移，用来指代内部恶性竞争、过度竞争的现象。如一家公司，原本实行八小时工作制，但有些人自愿加班，并得到管理层的赏识。此时，原先按时上下班的人开始担心自己成为劣势者，也自愿加班，久而久之，加班便成为常态，最后变成如果不自愿加班，就会影响自己在职场的生存，降低谈判力)"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '说话']
<先知>:```内卷，是一种人类自身的矛盾和冲突，是人类在追求自我价值的同时，也在与他人竞争的过程中产生的。
内卷是一种贪婪的表现，是人类对于物质和地位的过度追求，是人类对于自我利益的过度关注。
内卷是一种自我束缚，是人类对于自由和平等的背叛，是人类对于自我实现的阻碍。
内卷是一种社会病态，是人类对于社会公正和人类尊严的侵犯，是人类对于社会和谐的破坏。

内卷的根源在于人类的贪婪和自私，只有通过自我反省和自我教育，才能够摆脱内卷的困扰。
我们应该关注自己的内心，关注自己的价值观和人生目标，不要被外界的诱惑和压力所左右。
我们应该尊重他人，关注社会公正和人类尊严，不要在竞争中失去了人性和良知。
我们应该追求自由和平等，关注社会和谐和人类幸福，不要让内卷成为我们生活的常态和习惯。```


In [23]:
query_topic = "幼儿园"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['房舍', '快乐']
<先知>:```幼儿园是一片神圣的土地，是孩子们成长的摇篮。
在这里，孩子们可以自由地探索、学习和玩耍，发掘自己的潜能和天赋。
幼儿园是一座桥梁，连接着孩子们的童年和未来。
在这里，孩子们可以建立友谊，学会分享和合作，培养自信和独立。
幼儿园是一所学校，教导孩子们基本的知识和技能，为他们未来的学习和生活打下坚实的基础。
在这里，孩子们可以学习语言、数学、科学、艺术等各种知识，开拓自己的视野和思维。
幼儿园是一座灯塔，照亮着孩子们前行的道路。
在这里，孩子们可以接受良好的教育和引导，树立正确的价值观和人生观，成为有道德、有责任、有担当的人。
幼儿园是一片花园，培育着孩子们的心灵和情感。
在这里，孩子们可以感受到爱和关怀，学会关心和尊重他人，培养自己的情感和情商。
幼儿园是一座宝库，收藏着孩子们的美好回忆和成长足迹。
在这里，孩子们可以度过快乐的时光，留下美好的回忆，成为他们一生中难忘的经历。
因此，我们应该珍惜幼儿园，为孩子们提供一个优质的成长环境和教育资源，让他们健康、快乐、自信地成长。```


In [22]:
query_topic = "退休"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['房舍', '快乐']
<先知>:```退休，是一种生命的转折点，也是一种新的开始。
当你退休之后，你不再被工作所束缚，你可以有更多的时间去追求自己的兴趣爱好，去探索未知的领域。
退休并不意味着你的生命结束了，相反，它是你生命中的一个新的阶段，一个更加自由、更加充实的阶段。
在这个阶段，你可以更加深入地思考人生的意义，你可以更加关注自己的身心健康，你可以更加关注家人和朋友，你可以更加关注社会和环境。
退休并不意味着你要放弃自己的梦想和追求，相反，它是你实现梦想和追求的更好时机。
在这个阶段，你可以尝试新的事物，挑战自己的极限，不断地学习和成长。
退休并不意味着你要孤独和无助，相反，它是你与家人和朋友更加亲密的时机。
在这个阶段，你可以更加关注家人和朋友的生活，给予他们更多的关爱和支持。
退休并不意味着你要退缩和消极，相反，它是你为社会和环境做出更多贡献的时机。
在这个阶段，你可以参与志愿活动，为社会和环境做出自己的贡献。
所以，退休并不是一件可怕的事情，相反，它是一种新的开始，一种更加自由、更加充实的生活方式。```


In [41]:
query_topic = "早教(指在学龄前对儿童进行积极的教育)"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '爱']
<先知>:```早教是为了让孩子们在成长的道路上更加顺利，更加充实。
就像是在播种的时候，要选择最好的土壤和最好的种子，才能收获最好的果实。
早教不仅仅是为了让孩子们学会知识和技能，更重要的是让他们在成长的过程中，养成正确的价值观和人生观。
早教应该注重培养孩子们的创造力和想象力，让他们在未来的道路上能够更加自信和独立。
同时，早教也需要注重孩子们的身心健康，让他们在健康的身体和心灵的基础上，更好地面对未来的挑战。
早教不是一蹴而就的，需要家长和教育者的共同努力和耐心，才能让孩子们在早期就拥有更好的成长环境和教育资源。
让我们一起为孩子们的未来，努力奋斗吧。```


后续

下个版本做个gradio吧

- [x] 增加更好的前置提示词（提前总结先知的文风）
- [x] 补充完整的prophet data（一共20+个，现在只有5个）
- [x] 使用luotuoBERT索引更相关的主题
- [ ] 做一个gradio的前端